In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os

from cp2k_spm_tools import igor

In [ ]:
import matplotlib

class FormatScalarFormatter(matplotlib.ticker.ScalarFormatter):
    def __init__(self, fformat="%1.1f", offset=True, mathText=True):
        self.fformat = fformat
        matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,
                                                        useMathText=mathText)
    def _set_format(self, vmin, vmax):
        self.format = self.fformat
        if self._useMathText:
            self.format = '$%s$' % matplotlib.ticker._mathdefault(self.format)


def make_plot(fig, ax, data, extent, title=None, title_size=None, center0=False, vmin=None, vmax=None, cmap='gist_heat', noadd=False):
    if center0:
        data_amax = np.max(np.abs(data))
        im = ax.imshow(data.T, origin='lower', cmap=cmap, interpolation='bicubic', extent=extent, vmin=-data_amax, vmax=data_amax)
    else:
        im = ax.imshow(data.T, origin='lower', cmap=cmap, interpolation='bicubic', extent=extent, vmin=vmin, vmax=vmax)

    if noadd:
        ax.set_xticks([])
        ax.set_yticks([])
    else:
        ax.set_xlabel(r"x ($\AA$)")
        ax.set_ylabel(r"y ($\AA$)")
        #if 1e-3 < np.max(data) < 1e3:
        #    cb = fig.colorbar(im, ax=ax)
        #else:
        #    cb = fig.colorbar(im, ax=ax, format=FormatScalarFormatter("%.1f"))
        cb = fig.colorbar(im, ax=ax)
        cb.formatter.set_powerlimits((-2, 2))
        cb.update_ticks()
    if title:
        ax.set_title(title)
    if title_size:
        ax.title.set_fontsize(title_size)
    ax.axis('scaled')


def make_series_plot(fig, data, voltages):
    for i_bias, bias in enumerate(voltages):
        ax = plt.subplot(1, len(voltages), i_bias+1)
        make_plot(fig, ax, data[:, :, i_bias], title="V=%.2f"%bias, title_size=22, cmap='gist_heat', noadd=True)

In [ ]:
def make_label(info, index=None, homo_index=None):
    if info['type'] == 'const-height sts':
        label = 'ch-sts h=%.1f\n fwhm=%.2f' % (info['height'], info['fwhm'])
    elif info['type'] == 'const-height stm':
        label = 'ch-stm h=%.1f\n fwhm=%.2f' % (info['height'], info['fwhm'])
    elif info['type'] == 'const-isovalue sts':
        label = 'cc-sts isov=%.1e\n fwhm=%.2f' % (info['isovalue'], info['fwhm'])
    elif info['type'] == 'const-isovalue stm':
        label = 'cc-stm isov=%.1e\n fwhm=%.2f' % (info['isovalue'], info['fwhm'])

    elif info['type'] == 'const-height orbital':
        label = 'ch-orb h=%.1f' % info['height']
    elif info['type'] == 'const-isovalue orbital':
        label = 'cc-orb isov=%.1e' % info['isovalue']

    if index is not None and homo_index is not None:

        i_rel_homo = index - homo_index

        if i_rel_homo < 0:
            hl_label = "HOMO%+d" % i_rel_homo
        elif i_rel_homo == 0:
            hl_label = "HOMO"
        elif i_rel_homo == 1:
            hl_label = "LUMO"
        else:
            hl_label = "LUMO%+d" % (i_rel_homo-1)

        label += "\n"
        label += "MO %d, " % index + hl_label

    return label

# View stm

In [ ]:
npz_path = "/home/kristjan/work/_tk_cp2k-spm-tools/examples/benzene_stm/out/stm.npz"
loaded_data = np.load(npz_path, allow_pickle=True)

In [ ]:
loaded_data.files

In [ ]:
stm_general_info = loaded_data['stm_general_info'][()]
stm_series_info = loaded_data['stm_series_info']
stm_series_data = loaded_data['stm_series_data']

e_arr = stm_general_info['energies']
x_arr = stm_general_info['x_arr'] * 0.529177
y_arr = stm_general_info['y_arr'] * 0.529177

extent = [np.min(x_arr), np.max(x_arr), np.min(y_arr), np.max(y_arr)]

In [ ]:
figsize = (12, 3*(extent[3] - extent[2])/(extent[1] - extent[0]))

for i_e, e in enumerate(e_arr):
    fig, ax_arr = plt.subplots(1, 4, figsize=figsize)
    print("E = %.2f eV" % e)
    for i_ax, ax in enumerate(ax_arr):

        data = stm_series_data[i_ax]
        info = stm_series_info[i_ax]
        label = make_label(info)

        make_plot(fig, ax, data[i_e], extent, title=label, noadd=True)

    plt.show()

# View orbitals

In [ ]:
orb_path = "/home/kristjan/work/_tk_cp2k-spm-tools/examples/benzene_stm/out/orb.npz"
loaded_data = np.load(orb_path, allow_pickle=True)

In [ ]:
s0_orb_general_info = loaded_data['s0_orb_general_info'][()]
s0_orb_series_info = loaded_data['s0_orb_series_info']
s0_orb_series_data = loaded_data['s0_orb_series_data']

e_arr = s0_orb_general_info['energies']
x_arr = s0_orb_general_info['x_arr'] * 0.529177
y_arr = s0_orb_general_info['y_arr'] * 0.529177

extent = [np.min(x_arr), np.max(x_arr), np.min(y_arr), np.max(y_arr)]

In [ ]:
for info, data in zip(s0_orb_series_info, s0_orb_series_data):

    if info['type'] == 'const-height orbital' and info['height'] == 3.0:
        for i_orb in range(len(s0_orb_general_info['orb_indexes'])):

            print(e_arr[i_orb])

            label = make_label(
                info,
                s0_orb_general_info['orb_indexes'][i_orb],
                s0_orb_general_info['homo']
            )

            fig = plt.figure(figsize=(3, 3))
            ax = plt.gca()
            make_plot(fig, ax, data[i_orb], extent, cmap='seismic', center0=True, title=label, noadd=True)
            plt.show()

In [ ]:
s0_orb_series_data.shape

In [ ]:
igorwave = igor.Wave2d(
        data=s0_orb_series_data[0, 0],
        xmin=x_arr[0],
        xmax=x_arr[-1],
        xlabel='x [Angstroms]',
        ymin=y_arr[0],
        ymax=y_arr[-1],
        ylabel='y [Angstroms]',
)
igorwave.write("orbital.itx")

# Load and format .itx

In [ ]:
igor_file_path = "/home/kristjan/work/_proj_undecacene/FIGURES/FIG2/orb2_h6.0_s0_10lumo+0_e0.5906_ev.itx"

In [ ]:
igor_wave = igor.igor_wave_factory(igor_file_path)

In [ ]:
fig = plt.figure(figsize=(20, 4))
ax = plt.gca()

make_plot(fig, ax, igor_wave.data, igor_wave.extent,
          vmax=0.9*np.max(igor_wave.data),
          vmin=0.0, cmap='gist_gray')

plot_name = os.path.splitext(igor_file_path)[0]
plt.savefig(plot_name + ".png", dpi=300, bbox_inches='tight')
plt.show()